***Задание 1***

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [49]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [50]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [51]:
def classificator(row):
    if (row['rating'] <= 2) :
        return 'низкий рейтинг'
    elif (row['rating'] <=4) :
        return 'средний рейтинг'
    elif (row['rating'] > 4) :
        return 'высокий рейтинг'

In [52]:
ratings['class'] = ratings.apply(classificator, axis=1)
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


***Задание 2***

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [53]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] 
}
keywords = pd.read_csv('keywords.csv')

In [54]:
def geo_classificator(row):
    arr_keyword = row['keyword'].split(' ')
    for region,cities in geo_data.items():
        res = list(set(arr_keyword) & set(cities))
        if res:
            return region
    return 'undefined'

In [55]:
keywords['region'] = keywords.apply(geo_classificator, axis=1)

In [56]:
# проверяем работает ли
group_keywords = keywords.groupby('region').count().reset_index()
group_keywords.head()

,region,keyword,shows
0,undefined,99406,99406
1,Дальний Восток,64,64
2,Северо-Запад,146,146
3,Центр,384,384


***Задание 3***

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    2.1 для каждой строки пройдите по всем годам списка years
    
    2.2 если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    
    2.3 если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год


3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [57]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

In [58]:
years = list(range(1950,2011))

In [59]:
def production_year(row):
    for year in years:
        if row['title'].find(str(year)) != -1:
            return str(year)
    return '1900'

In [60]:
movies['year'] = movies.apply(production_year, axis=1)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [183]:
#считаем средний рейтинг по movieId

def groupby_function(data):
    return data.rating.mean()

avg_rating = ratings.groupby('movieId').apply(groupby_function).reset_index()
avg_rating.columns = ['movieId','rating']

def avg_rating_to_movie(row):
    for rat in avg_rating.itertuples():
        if getattr(rat, 'movieId') == row['movieId']:
            return getattr(rat, 'rating')

In [184]:
# добавляем средний рейтинг в dataframe и сортируем. Работает достаточно долго, 3-5 минут :) 
movies['avg_rating'] = movies.apply(avg_rating_to_movie, axis=1)

In [188]:
movies.sort_values(by=['avg_rating'], ascending=[False]).head(200)

,movieId,title,genres,year,avg_rating
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,1900,5.0
7321,71180,Padre padrone (1977),Drama,1977,5.0
6651,51471,Amazing Grace (2006),Drama|Romance,2006,5.0
6684,52617,Woman on the Beach (Haebyeonui yeoin) (2006),Comedy|Drama,2006,5.0
6726,53887,O Lucky Man! (1973),Comedy|Drama|Fantasy|Musical,1973,5.0
6739,54251,Dorian Blues (2004),Comedy,2004,5.0
6748,54328,My Best Friend (Mon meilleur ami) (2006),Comedy,2006,5.0
6807,55555,"Edge of Heaven, The (Auf der anderen Seite) (2...",Drama,2007,5.0
6860,56869,Drained (O cheiro do Ralo) (2006),Comedy,2006,5.0
6867,57038,To the Left of the Father (Lavoura Arcaica) (2...,Drama,2001,5.0
